# This notebook convert location column values to a geoson file because polygon data can't be used in Leaflet in R:

In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np
from shapely import wkt

In [2]:
### to see all columns
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [3]:
### Read the cleaned csv file
ecos_us_ca = pd.read_csv("../data/ecos_animal_us_ca_clean.csv")
ecos_us_ca.head()

,common_name,scientific_name,species_group,esa_status,foreign_or_domestic,image_url,image_url_full,location,country,food,habitat_requirements,home_range,cause,reproduction,description,status_code
0,Akekee,Loxops caeruleirostris,Birds,Endangered,Domestic,/docs/species_images/doc4660.png,https://ecos.fws.gov/docs/species_images/doc46...,"POLYGON((-160.30501096653 21.7282346819371,-16...",United States,"The 'Akeke'e feeds mostly on spiders, psyllid...",The 'Akeke'e inhabits Lowland Mesic and Wet Fo...,The 'Akeke'e is endemic to Kaua'i Island of th...,The 'Akeke'e population appears to have remain...,The 'Akeke'e nests in open-cup nests located ...,The 'Akeke'e is a small Hawaiian honeycreeper....,200
1,Polar bear,Ursus maritimus,Mammals,Threatened,Both Domestic and Foreign,/docs/species_images/doc5025.jpg,https://ecos.fws.gov/docs/species_images/doc50...,"POLYGON((-174.837571354155 59.7457496094217,-1...",Canada,No information available,No information available,No information available,No information available,No information available,Polar bears are protected under the Marine Mam...,200
2,Pitcher's thistle,Cirsium pitcheri,Flowering Plants,Threatened,Both Domestic and Foreign,/docs/species_images/doc5051.jpg,https://ecos.fws.gov/docs/species_images/doc50...,"POLYGON((-88.2001819788612 41.1621947025267,-8...",Canada,No information available,No information available,No information available,No information available,No information available,No information available,200
3,Piping Plover,Charadrius melodus,Birds,Endangered,Both Domestic and Foreign,/docs/species_images/doc3770.jpg,https://ecos.fws.gov/docs/species_images/doc37...,"POLYGON((-93.0885545621889 37.8619826666356,-9...",Canada,No information available,No information available,No information available,No information available,No information available,Size: 18 cm (7.25 in) in length. Color: Breedi...,200
4,Pink mucket (pearlymussel),Lampsilis abrupta,Clams,Endangered,Domestic,/docs/species_images/doc4977.jpg,https://ecos.fws.gov/docs/species_images/doc49...,"POLYGON((-93.8616440305422 32.505883021288,-93...",United States,Suspension feeder on plankton; deposit feeder ...,"Relatively silt-free substrates of sand, grave...",Sedentary.,No information available,"Separate sexes, males releasing sperm to water...",No information available,200


## 1- final cleaning before geospatial conversion:

### 1- Column "common_name" and "No common name" multiple values:

In [4]:
### There is 7 species with no common name and I need to modify it to be able to differenciate them for the app
### look at the rows numbers:
#ecos_us_ca.loc[:, "common_name"]

there are located in rows 57, 59, 60, 61, 62, 63 and 66

In [5]:
ecos_us_ca.iloc[65, :]

common_name                               Northeastern beach tiger beetle
scientific_name                               Cicindela dorsalis dorsalis
species_group                                                     Insects
esa_status                                                     Threatened
foreign_or_domestic                                              Domestic
image_url                                /docs/species_images/doc4860.jpg
image_url_full          https://ecos.fws.gov/docs/species_images/doc48...
location                POLYGON((-76.5301458997165 37.080589428058,-76...
country                                                     United States
food                    Adults are active predators that forage on sma...
habitat_requirements    Adult and larval tiger beetles are found on lo...
home_range              Once abundant along coastal beaches from Massa...
cause                   Few northeastern beach tiger beetle sites are ...
reproduction            Adults are pri

In [6]:
### Aristida chaseae
mask = (ecos_us_ca['common_name'] == "No common name") & (ecos_us_ca['scientific_name'] == "Aristida chaseae")
ecos_us_ca['common_name'][mask] = "Chase's threeawn"

<ipython-input-6-14f47d3c3a68>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ecos_us_ca['common_name'][mask] = "Chase's threeawn"


In [7]:
### Chamaecrista glandulosa var. mirabilis
mask = (ecos_us_ca['common_name'] == "No common name") & (ecos_us_ca['scientific_name'] == "Chamaecrista glandulosa var. mirabilis")
ecos_us_ca['common_name'][mask] = "Cassia mirabilis"

<ipython-input-7-acdd5d007732>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ecos_us_ca['common_name'][mask] = "Cassia mirabilis"


In [8]:
### Lepanthes eltoroensis
mask = (ecos_us_ca['common_name'] == "No common name") & (ecos_us_ca['scientific_name'] == "Lepanthes eltoroensis")
ecos_us_ca['common_name'][mask] = "Luquillo Mountain babyboot orchid"

<ipython-input-8-3807489729bb>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ecos_us_ca['common_name'][mask] = "Luquillo Mountain babyboot orchid"


In [9]:
### Lyonia truncata var. proctorii (there is really no common name so I change it for the scientific name otherwise I can't separate it with other no common name species)
mask = (ecos_us_ca['common_name'] == "No common name") & (ecos_us_ca['scientific_name'] == "Lyonia truncata var. proctorii")
ecos_us_ca['common_name'][mask] = "Lyonia truncata"

<ipython-input-9-010633ed5836>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ecos_us_ca['common_name'][mask] = "Lyonia truncata"


In [10]:
### Schoepfia arenaria (there is also no common name for this species so I change it for the scientific name
mask = (ecos_us_ca['common_name'] == "No common name") & (ecos_us_ca['scientific_name'] == "Schoepfia arenaria")
ecos_us_ca['common_name'][mask] = "Schoepfia arenaria"

<ipython-input-10-03d5a301fbbd>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ecos_us_ca['common_name'][mask] = "Schoepfia arenaria"


In [11]:
### Vernonia proctorii
mask = (ecos_us_ca['common_name'] == "No common name") & (ecos_us_ca['scientific_name'] == "Vernonia proctorii")
ecos_us_ca['common_name'][mask] = "Proctor's ironweed"

<ipython-input-11-620902dcd6f8>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ecos_us_ca['common_name'][mask] = "Proctor's ironweed"


In [12]:
### Cranichis ricartii
mask = (ecos_us_ca['common_name'] == "No common name") & (ecos_us_ca['scientific_name'] == "Cranichis ricartii")
ecos_us_ca['common_name'][mask] = "Puerto Rico helmet orchid"

<ipython-input-12-3795454425f0>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ecos_us_ca['common_name'][mask] = "Puerto Rico helmet orchid"


In [13]:
### Check if there is "No common name" values left:
ecos_us_ca[["common_name"]].eq("No common name").sum()

common_name    0
dtype: int64

### 2- Prepare dataframe for geo conversion:

In [14]:
### 436 rows
ecos_us_ca.shape

(434, 16)

In [15]:
### drop columns (I forgot to drop it before)
ecos_us_ca = ecos_us_ca.drop(["image_url", "status_code"], axis=1)

In [16]:
### There is 5 missing polygones so unfortunately I have to drop them... or I can't convert into geojson (other columns do not contain missing values).
ecos_us_ca = ecos_us_ca[ecos_us_ca["location"].notnull()]

In [17]:
### 429 rows left
ecos_us_ca.shape

(429, 14)

## 2- Df conversion to geojson files:

In [18]:
### The format of the polygon is WKT, so I have to convert it to shapely Polygon ("https://geopandas.readthedocs.io/en/latest/gallery/create_geopandas_from_pandas.html")
### Make a new column "geometry" for the polygon data
ecos_us_ca['geometry'] = ecos_us_ca["location"].apply(wkt.loads)

In [19]:
ecos_us_ca_geo = gpd.GeoDataFrame(ecos_us_ca)

In [20]:
ecos_us_ca_geo.set_geometry(col='geometry', inplace=True)

In [21]:
### Save the geodata
ecos_us_ca_geo.to_file("../data/ecos_us_ca_geo.geojson")